In [9]:
# Cell 1: Imports and Configuration

import os
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt # Keeping for potential future use or debugging, but not for main output
from skimage import color, filters
import random

# --- CONFIGURATION ---
# Path to your modified backgrounds folder
MODIFIED_BACKGROUNDS_DIR = "PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS" # <--- VERIFY THIS PATH

# Target size for processing (MUST MATCH the TARGET_SIZE you'll use for training)
# It's crucial that images are scaled to this size *before* applying filters,
# as sigma values are resolution-dependent.
TARGET_WIDTH = 2048 # <--- REPLACE WITH YOUR ACTUAL TARGET_WIDTH (e.g., 2048)
TARGET_HEIGHT = 2040 # <--- REPLACE WITH YOUR ACTUAL TARGET_HEIGHT (e.g., 2040)
TARGET_SIZE = (TARGET_WIDTH, TARGET_HEIGHT)

# Output directory for saved filter visualization images
FILTER_VISUALIZATIONS_DIR = os.path.join(MODIFIED_BACKGROUNDS_DIR, "FILTER_VISUALIZATIONS")
os.makedirs(FILTER_VISUALIZATIONS_DIR, exist_ok=True)


# --- Define Ridge Filter Parameters ---
# These can be a single float (e.g., [1.0]) or a list of floats (e.g., [1.0, 2.0, 3.0]).
# If a list is provided, skimage takes the maximum response across all scales.
sato_sigmas = [0.5, 1.0, 1.5, 2.0, 4.0]
meijering_sigmas = [0.5, 1.0, 1.5, 2.0, 4.0]
frangi_sigmas = [0.5, 1.0, 1.5, 2.0, 4.0]
hessian_sigmas = [0.5, 1.0, 1.5, 2.0, 4.0]

# --- Contrast Enhancement Parameter ---
# Percentile to clip and normalize image values for contrast enhancement
ENHANCE_PERCENTILE = 99.0 # Common values are 99.0 to 99.99

# Number of random images to process and save filter outputs for
NUM_IMAGES_TO_PROCESS = 1 # Changed to 1 to focus on detailed view per image

# --- Helper Functions (Copied from previous scripts, ensure they are up-to-date) ---

def rotate_to_wide(image_pil):
    """Rotates an image so its width is greater than its height."""
    width, height = image_pil.size
    if height > width:
        image_pil = image_pil.transpose(Image.Transpose.ROTATE_270)
    return image_pil

def rescale_and_pad_image(image_pil, target_size):
    """
    Rescales an image to fit within target_size while maintaining aspect ratio,
    then pads with white (for RGB) or black (for L) to reach target_size.
    """
    original_width, original_height = image_pil.size
    target_width, target_height = target_size
    
    scale_w = target_width / original_width
    scale_h = target_height / original_height
    scale_factor = min(scale_w, scale_h) # Use the smaller factor to 'fit' within the target size

    # Calculate new dimensions based on the common scale factor
    if original_width == 0 or original_height == 0:
        new_width = 0
        new_height = 0
    else:
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)

    scaled_img = image_pil.resize((new_width, new_height), Image.LANCZOS)

    paste_x = (target_width - new_width) // 2
    paste_y = (target_height - new_height) // 2
    
    if image_pil.mode == 'L':
        padded_img = Image.new("L", target_size, 0) # Use 0 for mask background
    else: # Assume RGB
        padded_img = Image.new("RGB", target_size, (255, 255, 255)) # Use white for RGB background
        
    padded_img.paste(scaled_img, (paste_x, paste_y))
    
    return padded_img

def enhance_contrast(arr, percentile_val):
    """Applies contrast enhancement based on percentile."""
    vmax = np.percentile(arr, percentile_val)
    if vmax == 0: # Avoid division by zero if all values are zero
        return np.zeros_like(arr, dtype=np.float32)
    arr_clipped = np.clip(arr, 0, vmax)
    arr_rescaled = arr_clipped / vmax
    return arr_rescaled.astype(np.float32) # Ensure float32 for consistency

def apply_ridge_filters_and_return_channels(image_pil_padded, sato_s, meijering_s, frangi_s, hessian_s, enhance_p):
    """
    Applies various ridge filters to a grayscale image and returns their enhanced outputs.
    Takes a PIL Image that is already padded to TARGET_SIZE.
    """
    image_rgb_float = np.array(image_pil_padded).astype(np.float32) / 255.0
    
    # If the image is RGB, convert to grayscale. If it's already L, it's fine.
    if image_rgb_float.ndim == 3 and image_rgb_float.shape[2] == 3:
        gray_image = color.rgb2gray(image_rgb_float)
    elif image_rgb_float.ndim == 2:
        gray_image = image_rgb_float
    else:
        raise ValueError(f"Unexpected image dimensions or mode for filter application: {image_rgb_float.shape}")

    # Apply filters using the provided sigma list (even if single value)
    sato_raw = filters.sato(gray_image, sigmas=sato_s, black_ridges=False, mode='reflect')
    meijering_raw = filters.meijering(gray_image, sigmas=meijering_s, black_ridges=False, mode='reflect')
    frangi_raw = filters.frangi(gray_image, sigmas=frangi_s, black_ridges=False, mode='reflect')
    hessian_raw = filters.hessian(gray_image, sigmas=hessian_s, black_ridges=True, mode='reflect') # Hessian typically black ridges

    # Enhance contrast using the provided percentile
    sato_processed = enhance_contrast(sato_raw, enhance_p)
    meijering_processed = enhance_contrast(meijering_raw, enhance_p)
    frangi_processed = enhance_contrast(frangi_raw, enhance_p)
    hessian_processed = enhance_contrast(hessian_raw, enhance_p)

    return sato_processed, meijering_processed, frangi_processed, hessian_processed


# Cell 2: Main Processing and Saving Logic

print(f"--- Saving Ridge Filter Outputs for Detailed Inspection ---")
print(f"Input Directory: {MODIFIED_BACKGROUNDS_DIR}")
print(f"Target Size for Processing: {TARGET_SIZE}")
print(f"Current Filter Sigmas: Sato={sato_sigmas}, Meijering={meijering_sigmas}, Frangi={frangi_sigmas}, Hessian={hessian_sigmas}")
print(f"Contrast Enhancement Percentile: {ENHANCE_PERCENTILE}")
print(f"Outputs will be saved to: {FILTER_VISUALIZATIONS_DIR}")


# Get list of all image files in the directory
all_image_files = [f for f in os.listdir(MODIFIED_BACKGROUNDS_DIR) if f.endswith(".png")]

if not all_image_files:
    print(f"Error: No PNG images found in '{MODIFIED_BACKGROUNDS_DIR}'. Please check the path and folder contents.")
else:
    # Randomly select images to process
    images_to_process = random.sample(all_image_files, min(NUM_IMAGES_TO_PROCESS, len(all_image_files)))

    for i, img_filename in enumerate(images_to_process):
        img_basename = os.path.splitext(img_filename)[0]
        img_path = os.path.join(MODIFIED_BACKGROUNDS_DIR, img_filename)
        img_pil = Image.open(img_path).convert("RGB") # Load as RGB for consistency

        # Preprocess the image to target size
        img_pil_preprocessed = rotate_to_wide(img_pil)
        img_pil_padded = rescale_and_pad_image(img_pil_preprocessed, TARGET_SIZE)
        
        # Apply filters
        sato_out, meijering_out, frangi_out, hessian_out = \
            apply_ridge_filters_and_return_channels(img_pil_padded, sato_sigmas, meijering_sigmas, frangi_sigmas, hessian_sigmas, ENHANCE_PERCENTILE)

        # Create a subfolder for the current image's filter outputs
        current_image_output_dir = os.path.join(FILTER_VISUALIZATIONS_DIR, img_basename)
        os.makedirs(current_image_output_dir, exist_ok=True)

        # Save each filter output as a separate PNG image
        filter_outputs = {
            "sato": sato_out,
            "meijering": meijering_out,
            "frangi": frangi_out,
            "hessian": hessian_out
        }

        print(f"\nProcessing and saving filters for: {img_basename}")
        for filter_name, data_array in filter_outputs.items():
            # Scale float [0,1] data to uint8 [0,255] for saving as image
            data_uint8 = (data_array * 255).astype(np.uint8)
            output_filepath = os.path.join(current_image_output_dir, f"{filter_name}_sigma={sato_sigmas if filter_name=='sato' else meijering_sigmas if filter_name=='meijering' else frangi_sigmas if filter_name=='frangi' else hessian_sigmas}_perc={ENHANCE_PERCENTILE}.png")
            
            # Use PIL to save the image
            Image.fromarray(data_uint8, mode='L').save(output_filepath) # Save as 'L' (grayscale)
            print(f"  Saved: {output_filepath}")

    print(f"\n--- Processing Complete ---")
    print(f"You can now find the individual filter output images in subfolders within: '{FILTER_VISUALIZATIONS_DIR}'")

--- Saving Ridge Filter Outputs for Detailed Inspection ---
Input Directory: PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS
Target Size for Processing: (2048, 2040)
Current Filter Sigmas: Sato=[0.5, 1.0, 1.5, 2.0, 4.0], Meijering=[0.5, 1.0, 1.5, 2.0, 4.0], Frangi=[0.5, 1.0, 1.5, 2.0, 4.0], Hessian=[0.5, 1.0, 1.5, 2.0, 4.0]
Contrast Enhancement Percentile: 99.0
Outputs will be saved to: PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS/FILTER_VISUALIZATIONS

Processing and saving filters for: 000_SYLVANER
  Saved: PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS/FILTER_VISUALIZATIONS/000_SYLVANER/sato_sigma=[0.5, 1.0, 1.5, 2.0, 4.0]_perc=99.0.png
  Saved: PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS/FILTER_VISUALIZATIONS/000_SYLVANER/meijering_sigma=[0.5, 1.0, 1.5, 2.0, 4.0]_perc=99.0.png
  Saved: PROCESSED_DATA_FOR_SEGMENTATION_VINIFERA/MODIFIED_BACKGROUNDS/FILTER_VISUALIZATIONS/000_SYLVANER/frangi_sigma=[0.5, 1.0, 1.5, 2.0, 4.0]